In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
!pip install face_recognition
!pip install docopt

In [ ]:
from re import T
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from datetime import datetime 


def take_photo(filename, quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(f"/content/drive/MyDrive/face_recognition/students_present_in_class/{filename}", 'wb') as f:
    f.write(binary)
  return filename


In [ ]:
import face_recognition
import cv2
import os
from google.colab.patches import cv2_imshow 
from sklearn import svm
from docopt import docopt
import numpy as np
from datetime import datetime
from IPython.display import Image
from datetime import date
import pytz

def read_img(path):
  img=cv2.imread(path)
  (h,w)=img.shape[:2]
  width=500
  ratio=width/float(w)
  height=int(ratio*h)
  return cv2.resize(img,(width, height))

def face_recognize():
    student_encoding=[]
    student_usn=[]
    date_today=date.today().strftime("%d-%m-%Y")
    student_images='/content/drive/MyDrive/face_recognition/students_photos'
    present_images=f"/content/drive/MyDrive/face_recognition/class_images"
    
    for file in os.listdir(student_images):
      student_img=read_img(student_images+'/'+file)
      #student_img=face_recognition.load_image_file(student_images+'/'+file)
      img_encode=face_recognition.face_encodings(student_img)[0]
      student_encoding.append(img_encode)
      student_usn.append(file.split('.')[0])
    face_names=[]
    for file in os.listdir(present_images):
        if ((file.split(":")[0])==(datetime.now().strftime("%Y-%m-%d %H"))):
          class_img=read_img(present_images+'/'+file)
          #class_img=face_recognition.load_image_file(present_images+'/'+file)
          face_locations = face_recognition.face_locations(class_img)
          no = len(face_locations)
          print("Number of faces detected: ", no)
          face_encodings=face_recognition.face_encodings(class_img,face_locations)
          for face_encoding in face_encodings:
              matches = face_recognition.compare_faces(student_encoding,face_encoding)
              face_distances = face_recognition.face_distance(student_encoding,face_encoding)
              name='Unknown'
              best_match_index = np.argmin(face_distances)
              if matches[best_match_index]:
                    name = student_usn[best_match_index]
              face_names.append(name)
    return face_names

In [84]:
import shutil
import os

def copy_file():
  present_images='/content/drive/MyDrive/face_recognition/students_present_in_class'
  destination=f"/content/drive/MyDrive/face_recognition/class_images"
  for file in os.listdir(present_images):
    shutil.move(present_images+'/'+file,destination)

def remove_img():
    present_images='/content/drive/MyDrive/face_recognition/students_present_in_class'
    for f in os.listdir(present_images):
      os.remove(present_images+'/'+f)

In [85]:
import pandas as pd
from datetime import datetime
from datetime import date
import calendar
import pytz

def create_usn_list():
  l=[]
  for i in range(71,144):
    if i<100:
      l.append(f"1NH20AI0{i}")
    else:
      l.append(f"1NH20AI{i}")
  return(l)

def present_absent(present_usn_list,usn_list,df,period):
  if df.empty:
      data=len(usn_list)*["Absent"]
      attendance=pd.DataFrame(data,columns=[period],index=usn_list)
  else:
      attendance=df
      attendance[period]=len(usn_list)*["Absent"]
  if period:
      for rowIndex, row in attendance.iterrows(): #iterate over rows
          for columnIndex, value in row.items():
              for usn in present_usn_list:
                if (rowIndex==usn) and (columnIndex==period):
                    attendance.at[rowIndex,columnIndex]="Present" 
  else:
    print("no period found")
  return(attendance)
  
def mark_attendance(df):
  b1_timetable={
      "9.00":["math","math","java","math","dbms"],
      "10.00":["ds","java","dbms","dbms lab","ds"],
      "11.10":["dbms","ds","math","dbms lab","java"],
      "12.10":["esa","dbms","ds","dbms lab","math"]
  }
  b2_timetable={
      "9.00":["math","math","java","math","dbms"],
      "10.00":["ds","java","dbms","java lab","ds"],
      "11.10":["dbms","ds","math","java lab","java"],
      "12.10":["esa","dbms","ds","java lab","math"]
  }
  
  days=["monday","tuesday","wednesday","thursday","friday"]
  tb1=pd.DataFrame(b1_timetable,index=days)
  tb2=pd.DataFrame(b2_timetable,index=days)
  students_usn=student_usn=create_usn_list()
  #data=[len(students_usn)*["Absent"],len(students_usn)*["Absent"],len(students_usn)*["Absent"],len(students_usn)*["Absent"]]
  present_usn=face_recognize()
  #weekday=calendar.day_name[date.today().weekday()]
  weekday="Thursday"
  timezone = pytz.timezone('Asia/Kolkata')
  #time=datetime.strftime(datetime.now(tz = timezone),"%H.%M")
  time="10.11"
  b1_classes=dict(tb1.loc[weekday.lower()])
  b2_classes=dict(tb2.loc[weekday.lower()])
  
  for item in b1_classes:
    if float(time)>=float(item):
      b1_period=b1_classes[item]
  for item in b1_classes:
    if float(time)>=float(item):
      b2_period=b2_classes[item]
  if weekday.lower()=="thursday":
    b1_usn_list=students_usn[0:(len(students_usn)//2)]
    b2_usn_list=students_usn[(len(students_usn)//2):]
    df_1 = df.loc[:(b1_usn_list[-1]),:]
    df_2 = df.loc[(b2_usn_list[0]):(b2_usn_list[-1]),:]
    at1=present_absent(present_usn,b1_usn_list,df_1,b1_period)
    at2=present_absent(present_usn,b2_usn_list,df_2,b2_period)
    attendance=pd.concat([at1,at2],axis=0)
  else:
    attendance=present_absent(present_usn,students_usn,df,b1_period)
  return attendance           
  

In [86]:
import pandas as pd
import os
def mark_attendance_on_excel():
  fpath="/content/drive/MyDrive/face_recognition/excel sheets"
  if(len(os.listdir("/content/drive/MyDrive/face_recognition/excel sheets"))==0):  
    path=fpath+"/"+datetime.strftime(datetime.today(),"%d-%m-%Y")+".xlsx"
    df=pd.DataFrame([])
    attendance=mark_attendance(df)
    attendance.to_excel(path)
  else:
    excelsheet_list=os.listdir("/content/drive/MyDrive/face_recognition/excel sheets")
    today=datetime.strftime(datetime.today(),"%d-%m-%Y")+".xlsx"
    if today in excelsheet_list:
        path=fpath+"/"+datetime.strftime(datetime.today(),"%d-%m-%Y")+".xlsx"
        df=pd.read_excel(path,sheet_name=0,index_col=0)
        attendance=mark_attendance(df)
        attendance.to_excel(path)  
    else:
        path=fpath+"/"+datetime.strftime(datetime.today(),"%d-%m-%Y")+".xlsx"
        df=pd.DataFrame([])
        attendance=mark_attendance(df)
        attendance.to_excel(path)

In [89]:
timezone = pytz.timezone('Asia/Kolkata')
datetime.strftime(datetime.now(tz = timezone),"%Y-%m-%d %H:%M:%S.%f")

'2022-06-30 00:06:41.714854'

In [87]:
import time
from datetime import datetime
mark=0
choice=int(input("Enter 1 if u want to click a photo else 0"))
timezone = pytz.timezone('Asia/Kolkata')
try:        
  while choice==1:
    filename=datetime.strftime(datetime.now(tz = timezone),"%Y-%m-%d %H:%M:%S.%f")+".jpg"
    take_photo(filename)
    time.sleep(10)
    copy_file()
    remove_img()
    time.sleep(10)
    choice=int(input("Enter 1 if u want to click a photo else 0"))
    if choice==0:
      mark=int(input("Enter 1 to mark attendance"))
  if mark==1:
    mark_attendance_on_excel()
    
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

Enter 1 if u want to click a photo else 01


<IPython.core.display.Javascript object>

Enter 1 if u want to click a photo else 00
Enter 1 to mark attendance1
Number of faces detected:  4
Number of faces detected:  3
Number of faces detected:  0
Number of faces detected:  1
Number of faces detected:  1
1
2
3
           dbms lab java lab
1NH20AI071   Absent      NaN
1NH20AI072  Present      NaN
1NH20AI073   Absent      NaN
1NH20AI074  Present      NaN
1NH20AI075   Absent      NaN
...             ...      ...
1NH20AI139      NaN   Absent
1NH20AI140      NaN   Absent
1NH20AI141      NaN   Absent
1NH20AI142      NaN   Absent
1NH20AI143      NaN   Absent

[73 rows x 2 columns]
